In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.datasets import make_blobs

In [ ]:
from torchsummary import summary
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
!git clone https://github.com/KuoYuChang/MLDL_video_course.git
%cd MLDL_video_course

In [ ]:
from utils.plot_tools import plot_dataset

### create dataset

In [ ]:
X, Y = make_blobs(n_samples=500, centers=2, random_state=0, cluster_std=0.4)
X = (X - X.mean()) / X.std()
#X = X + 10
Y[np.where(Y == 0)] = -1

In [ ]:
X.shape, Y.shape

In [ ]:
X = X.astype(np.float32)
Y = Y.astype(np.float32)

In [ ]:
plot_dataset(X, Y, size=20)

In [ ]:
# same dataset class af tensorflow playground
from utils.dataset_torch import playDataset

In [ ]:
batch_size = 16

In [ ]:
blobData = playDataset(Y, X)
blobLoader = DataLoader(blobData, batch_size=batch_size, shuffle=True)

In [ ]:
dataiter = iter(blobLoader)
print(next(dataiter))

### Primal form

In [ ]:
class SvmPrimal(nn.Module):
    def __init__(self, c=100, lr=0.001):
        super().__init__()

        self.c = c
        self.linear = nn.Linear(2, 1)
        
        
        # define loss, optimizer
        self.optimizer = optim.SGD(self.parameters(), lr=lr, momentum=0.9)

    def forward(self, x):
        x = self.linear(x)
        return x

    def get_loss(self, labels, predict, weight):
        

        return loss

    def train_step(self, inputs, labels):
        output = self(inputs)
        output = output.squeeze()

        weight = self.linear.weight.squeeze()

        input_loss = self.get_loss(labels, output, weight)

        input_loss.backward()

        self.optimizer.step()

        return input_loss.item()

In [ ]:
linear_svm = SvmPrimal(c=400)

In [ ]:
print(summary(linear_svm, input_size=(2,)))

In [ ]:
# training

num_epoch = 1000
currentLoader = blobLoader

for epoch in range(num_epoch):
    running_loss = 0.0

    count = 0
    for i, data in enumerate(currentLoader, 0):
        inputs, labels = data

        # zero the parameter gradients
        linear_svm.optimizer.zero_grad()

        batch_loss = linear_svm.train_step(inputs, labels)
        running_loss += batch_loss

        count = count + 1
        
    if epoch % 50 == 49:    # print every 2000 mini-batches
        #print("count: ", count)
        print(f'[{epoch + 1}] loss: {running_loss / count:.3f}')
        running_loss = 0.0
print('Finished Training')

In [ ]:
from utils.plot_tools import plot_model_contour

In [ ]:
plot_model_contour(linear_svm, X, Y, x_min=-2, x_max=2, size=20)

### reference
### https://github.com/kazuto1011/svm-pytorch?tab=readme-ov-file

### references
### https://gist.github.com/mlaves/c98cd4e6bcb9dbd4d0c03b34bacb0f65
### https://gist.github.com/mlaves
### https://github.com/adeveloperdiary/blog/blob/master/SVM/NonLinearSVM_DualProblem.py
### https://github.com/adeveloperdiary/blog/tree/master